<a href="https://colab.research.google.com/github/Plananas/AI-and-Data-Mining/blob/main/AI_and_Datamining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Optimizing Patient Flow: Predicting and Reducing Wait Times in Healthcare Appointments**



**Objective:**
Reduce patient wait times by analyzing historical data and identifying factors contributing to delays.
Develop a predictive model to estimate wait times based on key features.

Link to the data source: https://www.kaggle.com/datasets/carogonzalezgaltier/medical-appointment-scheduling-system

This program will be using the supplied data to reduce the wait times of patients.

Make sure this has been removed and changed before it is marked

In [1]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

#Import the data
appointments_file_path = "/content/drive/My Drive/Data/appointments.csv"
patients_file_path = "/content/drive/My Drive/Data/patients.csv"
slots_file_path = "/content/drive/My Drive/Data/slots.csv"

appointments = pd.read_csv(appointments_file_path)
patients = pd.read_csv(patients_file_path)
slots = pd.read_csv(slots_file_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
#List the info
appointments.info()
slots.info()
patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111488 entries, 0 to 111487
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   appointment_id        111488 non-null  int64  
 1   slot_id               111488 non-null  int64  
 2   scheduling_date       111488 non-null  object 
 3   appointment_date      111488 non-null  object 
 4   appointment_time      111488 non-null  object 
 5   scheduling_interval   111488 non-null  int64  
 6   status                111488 non-null  object 
 7   check_in_time         86032 non-null   object 
 8   appointment_duration  86032 non-null   float64
 9   start_time            86032 non-null   object 
 10  end_time              86032 non-null   object 
 11  waiting_time          86032 non-null   float64
 12  patient_id            111488 non-null  int64  
 13  sex                   111488 non-null  object 
 14  age                   111488 non-null  int64  
 15  

In [30]:
#Merge the tables together to create a single dataset that is easy to use.

full_dataset = appointments.merge(patients, on='patient_id', how='left')
full_dataset = full_dataset.merge(slots, on='slot_id', how='left')

#Appointments are the primary key as there can only ever be one
full_dataset.set_index('appointment_id', inplace=True)

full_dataset.info()


<class 'pandas.core.frame.DataFrame'>
Index: 111488 entries, 138 to 111488
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   slot_id               111488 non-null  int64         
 1   scheduling_date       111488 non-null  object        
 2   appointment_date_x    111488 non-null  datetime64[ns]
 3   appointment_time_x    111488 non-null  object        
 4   scheduling_interval   111488 non-null  int64         
 5   status                111488 non-null  object        
 6   check_in_time         86032 non-null   datetime64[ns]
 7   appointment_duration  86032 non-null   float64       
 8   start_time            86032 non-null   datetime64[ns]
 9   end_time              86032 non-null   datetime64[ns]
 10  waiting_time          86032 non-null   float64       
 11  patient_id            111488 non-null  int64         
 12  sex_x                 111488 non-null  object        
 13  ag

In [24]:
# Check missing values in the dataset
missing_values = full_dataset.isnull().sum()
print("Missing Values per Column:\n", missing_values)
# Check missing data distribution by status
full_dataset.groupby('status').apply(lambda x: x.isnull().sum())



Missing Values per Column:
 appointment_id              0
slot_id                     0
scheduling_date             0
appointment_date_x          0
appointment_time_x          0
scheduling_interval         0
status                      0
check_in_time           25456
appointment_duration    25456
start_time              25456
end_time                25456
waiting_time            25456
patient_id                  0
sex_x                       0
age                         0
age_group                   0
name                        0
sex_y                       0
dob                         0
insurance                   0
appointment_date_y          0
appointment_time_y          0
is_available                0
dtype: int64


<ipython-input-24-088046e7b92d>:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  full_dataset.groupby('status').apply(lambda x: x.isnull().sum())


,appointment_id,slot_id,scheduling_date,appointment_date_x,appointment_time_x,scheduling_interval,status,check_in_time,appointment_duration,start_time,...,sex_x,age,age_group,name,sex_y,dob,insurance,appointment_date_y,appointment_time_y,is_available
status,,,,,,,,,,,,,,,,,,,,,
attended,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cancelled,0,0,0,0,0,0,0,18254,18254,18254,...,0,0,0,0,0,0,0,0,0,0
did not attend,0,0,0,0,0,0,0,6615,6615,6615,...,0,0,0,0,0,0,0,0,0,0
scheduled,0,0,0,0,0,0,0,141,141,141,...,0,0,0,0,0,0,0,0,0,0
unknown,0,0,0,0,0,0,0,446,446,446,...,0,0,0,0,0,0,0,0,0,0


We should remove the information of appointments that either cancelled, didnt attend, schedules, or are unknown as they are not appropriate to the analysis.

In [32]:
# Keep only attended appointments
appointments_df = appointments[appointments['status'] == 'Attended']


In [38]:
# Ensure dataset is loaded correctly
# appointments_df = pd.read_csv("<your_file>.csv")

# Convert relevant columns from string to datetime
# FIXME make this work properly
appointments['appointment_datetime'] = pd.to_datetime(appointments['appointment_date'].astype(str) + ' ' + appointments['appointment_time'].astype(str), errors='coerce')
appointments['check_in_datetime'] = pd.to_datetime(appointments['appointment_date'].astype(str) + ' ' + appointments['check_in_time'].astype(str), errors='coerce')
appointments['start_datetime'] = pd.to_datetime(appointments['appointment_date'].astype(str) + ' ' + appointments['start_time'].astype(str), errors='coerce')
appointments['end_datetime'] = pd.to_datetime(appointments['appointment_date'].astype(str) + ' ' + appointments['end_time'].astype(str), errors='coerce')


<ipython-input-38-a0c898b7a55f>:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  appointments['check_in_datetime'] = pd.to_datetime(appointments['appointment_date'].astype(str) + ' ' + appointments['check_in_time'].astype(str), errors='coerce')
<ipython-input-38-a0c898b7a55f>:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  appointments['start_datetime'] = pd.to_datetime(appointments['appointment_date'].astype(str) + ' ' + appointments['start_time'].astype(str), errors='coerce')
<ipython-input-38-a0c898b7a55f>:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  appointments['e

KeyboardInterrupt: 

In [37]:
# probably combine the date and time values just incase it was longer than one day or at midnight


#FIXME this needs to use the full datetimes calculated
# Calculate waiting time in minutes
appointments['waiting_time'] = (appointments['start_time'] - appointments['check_in_time']).dt.total_seconds() / 60

# Calculate delay in minutes
appointments['delay_from_scheduled'] = (appointments['start_time'] - appointments['appointment_time']).dt.total_seconds() / 60



<ipython-input-37-16a267adf2eb>:7: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  appointments['delay_from_scheduled'] = (appointments['start_time'] - appointments['appointment_time']).dt.total_seconds() / 60


TypeError: unsupported operand type(s) for -: 'NaTType' and 'datetime.time'

Unneeded Columns:

* Sex (two fields for it)
* Name (personal unique info not needed)
* dob (replace with age)
* scheduling date (we are only focused on the wait time)
* status (all the data has come from attended appointments)


